<a href="https://colab.research.google.com/github/KellyJBelly/marketing_analytics-case3/blob/main/MarketingAnalytics_CaseStudy3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study 2 :  John Rhees, Tommy Braswell, Annabelle Cunningham, & Kelly Fisher

## Hypothesis Questions:


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
# GITHUB_BASE = "https://github.com/KellyJBelly/marketing_analytics-case3"

# Raw GitHub URLs (replace with your actual raw links)
cpi_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/main/CPIUFDSL.csv"
diesel_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/main/GASDESM.csv"
expected_inflation_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/main/EXPINF10YR.csv"
unemployment_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/main/file.csv"
meat_grain_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/refs/heads/main/Meat_Grain.csv"
meat_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/refs/heads/main/Meaty.csv"
grain_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/refs/heads/main/Grainy.csv"
food_cpi_url = "https://raw.githubusercontent.com/KellyJBelly/marketing_analytics-case3/refs/heads/main/food_cpi_sets.csv"

# Load CSVs
cpi = pd.read_csv(cpi_url, parse_dates=['observation_date'])
diesel = pd.read_csv(diesel_url, parse_dates=['observation_date'])
expected_inflation = pd.read_csv(expected_inflation_url, parse_dates=['observation_date'])
unemp = pd.read_csv(unemployment_url)

# Parse unemployment date and filter columns
unemp['Month'] = unemp['Period'].str[1:].astype(int)
unemp['Date'] = pd.to_datetime(unemp['Year'].astype(str) + '-' + unemp['Month'].astype(str) + '-01')
unemp = unemp[['Date', 'Value']].rename(columns={'Value': 'Unemployment_rate'})

# Rename columns for consistency
cpi = cpi.rename(columns={'observation_date': 'Date', 'CPIUFDSL': 'CPI_food'})
diesel = diesel.rename(columns={'observation_date': 'Date', 'GASDESM': 'Diesel_price'})
expected_inflation = expected_inflation.rename(columns={'observation_date': 'Date', 'EXPINF10YR': 'Inf_Exp_10yr'})

# Merge all datasets on 'Date'
df = cpi.merge(diesel, on='Date') \
        .merge(expected_inflation, on='Date') \
        .merge(unemp, on='Date')

In [ ]:
#MEAT ONLY
# meaty = pd.read_csv(meat_url)
# meaty.head()

#GRAIN ONLY
# grainy = pd.read_csv(grain_url)
# grainy.head()


#MEAT & GRAIN COMBIED
# meaty_grain = pd.read_csv(meat_grain_url)
# meaty_grain.head()

In [ ]:
food_cpi = pd.read_csv(food_cpi_url)
food_cpi.head()

In [ ]:
# Display head and date range
print(df[['Date', 'CPI_food', 'Diesel_price', 'Inf_Exp_10yr', 'Unemployment_rate']].head(3))
print("Date range:", df['Date'].min(), "to", df['Date'].max())

## Exploratory Data Analysis & Data Transformation

In [ ]:
# Sort by date and compute % changes
df = df.sort_values('Date')
df['Food_Percentage_Change']   = df['CPI_food'].pct_change() * 100
df['Diesel_Percent_Change'] = df['Diesel_price'].pct_change() * 100
df['InfExp_pct_chg'] = df['Inf_Exp_10yr'].pct_change() * 100
df.dropna(inplace=True)

# Summary statistics
print(df[['Food_Percentage_Change', 'Diesel_Percent_Change', 'InfExp_pct_chg']].describe())

# Time series plots
plt.figure(figsize=(14, 6))
plt.plot(df['Date'], df['Food_Percentage_Change'], label='Food Price % Change')
plt.plot(df['Date'], df['Diesel_Percent_Change'], label='Diesel Price % Change', alpha=0.7)
plt.plot(df['Date'], df['InfExp_pct_chg'], label='Inflation Expectations % Change', alpha=0.7)
plt.legend()
plt.title('Monthly % Changes Over Time')
plt.xlabel('Date')
plt.ylabel('Percent Change')
plt.grid(True)
plt.show()

# Pairplot for correlation visualization
sns.pairplot(df[['Food_Percentage_Change', 'Diesel_Percent_Change', 'InfExp_pct_chg']])
plt.suptitle("Pairwise Relationships Between Variables", y=1.02)
plt.show()

# Correlation matrix heatmap
corr = df[['Food_Percentage_Change', 'Diesel_Percent_Change', 'InfExp_pct_chg']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

## OLS Regression: Food % Change ~ Diesel % Change + Inflation Expectation % Change

In [ ]:
import statsmodels.api as sm

# Prepare data for regression
X = df[['Diesel_Percent_Change', 'InfExp_pct_chg']]
X = sm.add_constant(X)  # add constant term
y = df['Food_Percentage_Change']

# Fit OLS model
model = sm.OLS(y, X).fit()
print(model.summary())

#Augmented Dickey Fuller Stationarity Test
residuals = model.resid
adf_result = adfuller(residuals)
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")


## Casual Delay Affect

This is relative to real world economic systems, causal effects often occur with a time lag rather than instantaneously. For example, when diesel prices increase, transportation and logistics costs for food suppliers go up — but it may take weeks or months for those costs to be reflected in retail grocery prices due to existing inventory, supply contracts, and pricing cycles. This delay is known as a causal delay effect, and it's crucial in time series modeling because ignoring it can lead to underestimating true relationships between variables. Including lagged variables helps capture these delayed pass-through effects and improves both explanatory power and forecast accuracy.

In [ ]:
# add lags to show impact of time lag as it is relative to real world
df['Diesel_Percent_Change_lag1'] = df['Diesel_Percent_Change'].shift(1)
df['InfExp_pct_chg_lag1'] = df['InfExp_pct_chg'].shift(1)

# OLS regression with lags
X_lagged = sm.add_constant(df[['Diesel_Percent_Change', 'Diesel_Percent_Change_lag1',
                               'InfExp_pct_chg', 'InfExp_pct_chg_lag1']].dropna())
y_lagged = df['Food_Percentage_Change'].loc[X_lagged.index]

model_lag = sm.OLS(y_lagged, X_lagged).fit()
print(model_lag.summary())

#Feedback Loops Created by Diesel Pricing Vector Auto-Regression

In [ ]:
var_data = df[['Food_Percentage_Change', 'Diesel_Percent_Change', 'InfExp_pct_chg']].dropna()

# Fit VAR model with lag selection based on AIC
model_var = VAR(var_data)
results_var = model_var.fit(maxlags=4, ic='aic')
print(results_var.summary())

# Impulse Response Plot (12-period horizon)
irf = results_var.irf(12)

# Fix overlapping text with larger figure and better layout
fig = irf.plot(orth=False, figsize=(14, 10))
fig.suptitle("Impulse Response Functions (12 Months)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])  # reserve space for title
plt.show()


## Consumer Price Index Food

In [ ]:
import pandas as pd
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

# STEP 1: Load the actual data (skip the first 2 rows: metadata)
food_prices = pd.read_csv(food_cpi_url, skiprows=2)

# STEP 2: Rename the 'Frequency' column to 'Period' and convert to datetime
food_prices.rename(columns={'Frequency': 'Period'}, inplace=True)

# Convert 'Period' like '1993M7' to datetime
def convert_period(period_str):
    year = int(period_str[:4])
    month = int(period_str[5:])
    return pd.to_datetime(f"{year}-{month:02d}-01")

food_prices['Date'] = food_prices['Period'].apply(convert_period)

# STEP 3: Drop the metadata columns like 'Data Type' or 'USD' if present
food_prices = food_prices[~food_prices['Date'].isna()]  # remove metadata rows
food_prices.drop(columns=['Period'], inplace=True)

# STEP 4: Set datetime as index and sort
food_prices.set_index('Date', inplace=True)
food_prices.sort_index(inplace=True)

# STEP 5: Merge with macroeconomic dataframe
df['Date'] = pd.to_datetime(df['Date'])
merged_df = food_prices.merge(df, on='Date', how='inner')

# ✅ STEP 6: Compute percent change and clean NaNs/Infs
import numpy as np

# Only use numeric columns (drop any non-numeric metadata like 'Date')
merged_pct_df = merged_df.select_dtypes(include='number').pct_change() * 100

# Clean out inf and NaN values
merged_pct_df.replace([np.inf, -np.inf], np.nan, inplace=True)
merged_pct_df.dropna(inplace=True)

# (Optional) check if cleaning worked
print("NaNs left:", merged_pct_df.isna().sum().sum())
print("Infs left:", np.isinf(merged_pct_df.values).sum())

# STEP 7: Fit VAR model
model = VAR(merged_pct_df)
results = model.fit(maxlags=4, ic='aic')
print(results.summary())

# STEP 8: Plot IRFs
irf = results.irf(12)
fig = irf.plot(orth=False, figsize=(14, 10))
fig.suptitle("Impulse Response Functions (VAR Model)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


NaNs left: 0
Infs left: 0


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 17, Apr, 2025
Time:                     03:13:33
--------------------------------------------------------------------
No. of Equations:         23.0000    BIC:                    114.727
Nobs:                     291.000    HQIC:                   110.550
Log likelihood:          -24623.8    FPE:                6.34894e+46
AIC:                      107.759    Det(Omega_mle):     1.02588e+46
--------------------------------------------------------------------
Results for equation Monthly
                                   coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------------------------
const                                 0.093022         0.364603            0.255           0.799
L1.Monthly                            0.295755         0.093752            3.155           

<ipython-input-220-95eaa96cee24>:54: UserWarning: Tight layout not applied. tight_layout cannot make Axes width small enough to accommodate all Axes decorations
  plt.tight_layout(rect=[0, 0, 1, 0.96])


In [ ]:

# === Correlation Heatmap =====
plt.figure(figsize=(12, 8))
sns.heatmap(merged_pct_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix of % Changes", fontsize=14)
plt.show()

# === Time Series Preview of Key Variables ===
plt.figure(figsize=(14, 6))
for col in ['Beef', 'Corn', 'Diesel_price']:  # modify this list based on whatwe wanna show
    if col in merged_pct_df.columns:
        plt.plot(merged_pct_df.index, merged_pct_df[col], label=col)
plt.legend()
plt.title("Selected % Changes Over Time")
plt.xlabel("Date")
plt.ylabel("Percent Change")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# === Forecast Plot (next 12 periods) ===
forecast = results.forecast(merged_pct_df.values[-results.k_ar:], steps=12)
forecast_df = pd.DataFrame(forecast, index=pd.date_range(start=merged_pct_df.index[-1], periods=12, freq='MS'),
                           columns=merged_pct_df.columns)

# Plot forecast for a few key variables
plt.figure(figsize=(14, 6))
for col in ['Beef', 'Corn', 'Diesel_price']:  # modify this list
    if col in forecast_df.columns:
        plt.plot(forecast_df.index, forecast_df[col], label=f'Forecast: {col}')
plt.legend()
plt.title("12-Month VAR Forecast")
plt.xlabel("Date")
plt.ylabel("Forecasted % Change")
plt.grid(True)
plt.tight_layout()
plt.show()

## Meat - Food Index Vector Autoregression

## Grain - Food Index Vector Autoregression


## Price Sensitivity from Economic Events (COVID, 2008 Financial Crisis)

In [ ]:
df['Era'] = df['Date'].apply(lambda x: 'Post2020' if x >= pd.to_datetime("2020-01-01") else 'Pre2020')
sns.boxplot(x='Era', y='Food_Percentage_Change', data=df)
plt.title("Grocery Inflation Before and After 2020")
plt.show()


df['Era'] = df['Date'].apply(lambda x: 'Financial Crisis of 2008' if x >= pd.to_datetime("2010-07-01") else 'Pre2008')
sns.boxplot(x='Era', y='Food_Percentage_Change', data=df)
plt.title("Grocery Inflation Before and After 2008 Financial Crisis")
plt.show()